DataSet Builder Tutorial
==

The aim of this tuto is to provide an E2E example for a simple query. We will use a reduced dataset provided by SWH and apply our approach as it was the entire graph.
>the first 3k popular repositories tagged as being written in the Python language, from GitHub, Gitlab, PyPI and Debian 

The correponding Date is 2021-03-23, we will take 2021-02-23 as the fingerprint timestamp.
𝑡x=2021-03-23

We consider the following query q as 
𝑞 = "Get All the Origin having at least one file named "README.md" on its file hiearchy.

The corresponding fingerprint is the following : 
𝐹𝑃tx = ⟨𝑡x, 𝑞⟩


#### 0. Install the required dependencies 

The current workdir should be the root of the repository, if it's not the case run the next query

In [4]:
%cd ..

/home/creux/Documents/Software Heritage/code/DatasetBuilder


In [5]:
!sh install.sh

Installing dsi-utils
[INFO] Scanning for projects...
[INFO] 
[INFO] ------------------< org.apache.maven:standalone-pom >-------------------
[INFO] Building Maven Stub Project (No POM) 1
[INFO] --------------------------------[ pom ]---------------------------------
[INFO] 
[INFO] --- maven-install-plugin:2.4:install-file (default-cli) @ standalone-pom ---
[INFO] Installing /home/creux/Documents/Software Heritage/code/DatasetBuilder/thirdPartyLibrary/dsiutils/dsiutils-2.7.3-threadSafe.jar to /home/creux/.m2/repository/it/unimi/dsi/dsiutils/2.7.3-threadSafe/dsiutils-2.7.3-threadSafe.jar
[INFO] Installing /home/creux/Documents/Software Heritage/code/DatasetBuilder/thirdPartyLibrary/dsiutils/pom.xml to /home/creux/.m2/repository/it/unimi/dsi/dsiutils/2.7.3-threadSafe/dsiutils-2.7.3-threadSafe.pom
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total t

#### 1. Download the graphs

Install aws s3 to download the graph , https://aws.amazon.com/cli/

In [7]:
!pip3 install awscli --upgrade --user

In [ ]:
!mkdir ./tutorial/graph
!aws s3 cp --no-sign-request s3://softwareheritage/graph/2021-03-23-popular-3k-python/orc/origin_visit_status/ ./tutorial/graph/2021-03-23-popular-3k-python/orc/origin_visit_status --recursive
!aws s3 cp --no-sign-request s3://softwareheritage/graph/2021-03-23-popular-3k-python/compressed/ ./tutorial/graph/2021-03-23-popular-3k-python/compressed --recursive

#### 2. Design your ocl query

Design your query based on the template, here we are selecting all the repository containing a "README.md" file on their master or main branch. In the tutorial we will use the query saved in ```myquery.ocl```. Basically you can create any ocl query that applies on the SWH-Graph UML model.

<img src="image.png" alt="drawing" style="width:700px;"/>

In [6]:
%%file ./tutorial/myquery.ocl
import swhModel : 'platform:/resource/fr.inria.diverse.swhModel/model/swhModel.ecore'
package swhModel
context Graph
def : query():Set(Snapshot) = origins->select(
	 getLastSnapshot().branches->exists(
	 				(name= 'refs/heads/master' or name= 'refs/heads/main')
	 				and
					/*The main branch contains a file 'README.md'*/
					getRevision().tree.entries->closure(entry:DirectoryEntry |
						if entry.child.oclIsKindOf(Directory) then
							entry.child.oclAsType(Directory).entries.oclAsSet()
						else 
							entry.oclAsSet()
						endif	
					)->exists(e:DirectoryEntry | e.name='README.md')	
	)
)


endpackage


Overwriting ./tutorial/myquery.ocl


To have LSP support you can use the gemoc studio by importing the 
```fr.inria.diverse.swhModel.queryExemple```
and
```fr.inria.diverse.swhModel```

then modify the template.ocl

NB: The completion is not fully functional

### 3. Compile your query 

In [7]:
!sh ./oclQueryCompiler.sh ./tutorial/myquery.ocl RUNNING_QUERY ./tutorial

Launching generation for the model : ./tutorial/myquery.ocl, output path : ./tutorial/GraphQuery.java ,query id : RUNNING_QUERY
Model loaded, starting generation...
Generation done, saving results in ./tutorial/GraphQuery.java
Result saved


Change the GraphQuery file

In [8]:
!sh ./oclQueryCopyPackager.sh ./tutorial/myquery.ocl RUNNING_QUERY ./tutorial

Copying the produce ./tutorial/GraphQuery.java in ./fr.inria.diverse.fingerprint/src/main/java/fr/inria/diverse/query/GraphQuery.java 
Building fr.inria.diverse.fingerprint
[INFO] Scanning for projects...
[INFO] 
[INFO] -----------< fr.inria.diverse:fr.inria.diverse.fingerprint >------------
[INFO] Building fr.inria.diverse.fingerprint 0.1.0
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:3.1.0:clean (default-clean) @ fr.inria.diverse.fingerprint ---
[INFO] Deleting /home/creux/Documents/Software Heritage/code/DatasetBuilder/./fr.inria.diverse.fingerprint/target
[INFO] 
[INFO] --- maven-resources-plugin:3.3.0:copy-resources (copy-resources) @ fr.inria.diverse.fingerprint ---
[INFO] Copying 1 resource
[INFO] 
[INFO] --- maven-resources-plugin:3.3.0:resources (default-resources) @ fr.inria.diverse.fingerprint ---
[INFO] Copying 1 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.0:compile (default-compile) @ fr.inr

Now you can observe a jar in your workspace ```fr.inria.diverse.fingerprint-0.1.0.jar```

### Execute your query

Let's have a look on the help 

In [39]:
!java -jar tutorial/fr.inria.diverse.fingerprint-0.1.0.jar --help

[picocli WARN] defaults configuration file /home/rlefeuvr/Workspaces/SAND_BOX/test_swh/DatasetBuilder/./config/config.properties does not exist or is not readable
Usage: GraphQueryRunner [-h] [-c=<checkPointIntervalInMinutes>]
                        [-e=<exportPath>] -g=<graphFolderPath>
                        [-l=<loadingMode>] [-qt=<queryTimestamp>]
                        [-t=<threadNumber>]
Execute a query over the graph property dataset
  -c, --checkPointIntervalInMinutes=<checkPointIntervalInMinutes>
               The time in minutes after which a checkpoint will be produced
  -e, --exportPath=<exportPath>
               The export path, where all the queries results will be saved
                 including checkpoints
  -g, --graphPath=<graphFolderPath>
               The graph Folder path
  -h, --help   display this help and exit
  -l, --loadingMode=<loadingMode>
               The graph loading mode either MAPPED for memory mapped or RAM
                 for ram loading
   

In [10]:
!java -XX:PretenureSizeThreshold=512M -Djava.io.tmpdir=../java-tmp-dir -Xmx10G --add-opens=java.base/sun.nio.ch=ALL-UNNAMED -jar ./tutorial/fr.inria.diverse.fingerprint-0.1.0.jar  -g ./tutorial/graph/2021-03-23-popular-3k-python -l MAPPED -t 14 --queryTimestamp=2021-03-23 --exportPath=./tutorial/result --checkPointIntervalInMinutes=1

[picocli WARN] defaults configuration file /home/creux/Documents/Software Heritage/code/DatasetBuilder/./config/config.properties does not exist or is not readable
15:49:33.983 [main] INFO  fr.inria.diverse.query.GraphQueryRunner - Configuration [getThreadNumber()=14, getGraphPath()=./tutorial/graph/2021-03-23-popular-3k-python/compressed/graph, getRelationalPath()=./tutorial/graph/2021-03-23-popular-3k-python/orc, getGraphTimestamp()=2021-03-22T00:00+01:00[Europe/Paris], getQueryTimestamp()=2021-03-23T00:00+01:00[Europe/Paris], getGraphFolderPath()=./tutorial/graph/2021-03-23-popular-3k-python, getLoadingMode()=MAPPED, getExportPath()=./tutorial/result, getCheckPointIntervalInMinutes()=1]
15:49:33.998 [main] INFO  fr.inria.diverse.Graph - Loading graph MAPPED MODE
15:49:36.608 [main] INFO  fr.inria.diverse.Graph - Loading CommitterTimestamps
15:49:36.610 [main] INFO  fr.inria.diverse.Graph - CommitterTimestamps Loaded
15:49:36.610 [main] INFO  fr.inria.diverse.Graph - Graph loaded
15:

In [44]:
##Todo Investigate the exception raised for one origin, no impact on the others

You can now process the results, origin_url_snap_swhid.json contains the list of origins url that match the query as well as the snapshots ids present in the current version of the graph 